In [26]:
from torch import nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.optim import SGD

def acc(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

class all_con(pl.LightningModule):
    def __init__(self):
        super(all_con,self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5, stride=1, padding=0)
        self.bnorm1 = nn.BatchNorm2d(6)
        self.max1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0)
        self.bnorm2 = nn.BatchNorm2d(16)
        self.max2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.f = nn.Flatten()
        self.li = nn.Linear(13456, 900)
        self.li1 = nn.Linear(900, 450)
        self.li2 = nn.Linear(450, 101)
        self.lr = 0.01
        self.loss = nn.CrossEntropyLoss()
        
           
    def forward(self, x):
        batch_size, _, _, _ = x.size()
        x = self.conv1(x)
        x = F.relu(self.bnorm1(x))
        x = self.max1(x)
        x = self.conv2(x)
        x = F.relu(self.bnorm2(x))
        x = self.max2(x)
        x = self.f(x)
        x = F.relu(self.li(x))
        x = F.relu(self.li1(x))
        x = self.li2(x)
        return x
    
    def configure_optimizers(self):
        # Define Optimizer Here
        return SGD(self.parameters(),lr = self.lr)
       
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        y_hat = self.forward(x)
        loss = self.loss(y_hat,y)
        self.log("train_loss", loss)
        return loss
        
    def validation_step(self, valid_batch, batch_idx):
        x, y = valid_batch
        y_hat = self.forward(x)
        loss = self.loss(y_hat,y)
        
        pred = acc(y_hat, y)
        self.log("val_accuracy", pred[0])
        self.log("val_loss", loss)
        return loss
    
    def test_step(self, test_batch, batch_idx):
        l_test=[]
        
        x, y = test_batch
        y_hat = self.forward(x)
        loss = self.loss(y_hat,y)

        l_test.append(loss)
        
        pred = acc(y_hat, y)

        
        self.log("test_accuracy", pred[0])
        self.log("test_loss", loss)
        return l_test[-1]


In [27]:
import pytorch_lightning as pl
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
from torchvision.datasets import Food101
import torch
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization

class Food101(pl.LightningDataModule):
    def prepare_data(self):
        transform=transforms.Compose([
            transforms.RandomResizedCrop(128),
            transforms.RandomCrop(128),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5],
                                 [0.229, 0.224, 0.225])])
      
        self.train_data = datasets.Food101('C/Users/jasti/Downloads/data', split="train", download=True, transform=transform)
        self.val_data = datasets.Food101('C/Users/jasti/Downloads/data', split="test", download=True, transform=transform)
        self.test_data = datasets.Food101('C/Users/jasti/Downloads/data', split="test", download=True, transform=transform)
        print('[INFO] data loaded successfully')

    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size= 32, shuffle=True, num_workers=8)

    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size= 32, shuffle=True, num_workers=8)
    
    def test_dataloader(self):
        return DataLoader(self.test_data, batch_size= 32, shuffle=True, num_workers=8)


In [28]:
model = all_con()
Food101 = Food101()
trainer = pl.Trainer(accelerator ="cpu", max_epochs=5)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [29]:
trainer.fit(model, Food101)

INFO:pytorch_lightning.callbacks.model_summary:
   | Name   | Type             | Params
---------------------------------------------
0  | conv1  | Conv2d           | 456   
1  | bnorm1 | BatchNorm2d      | 12    
2  | max1   | MaxPool2d        | 0     
3  | conv2  | Conv2d           | 2.4 K 
4  | bnorm2 | BatchNorm2d      | 32    
5  | max2   | MaxPool2d        | 0     
6  | f      | Flatten          | 0     
7  | li     | Linear           | 12.1 M
8  | li1    | Linear           | 405 K 
9  | li2    | Linear           | 45.6 K
10 | loss   | CrossEntropyLoss | 0     
---------------------------------------------
12.6 M    Trainable params
0         Non-trainable params
12.6 M    Total params
50.261    Total estimated model params size (MB)


[INFO] data loaded successfully


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [30]:
trainer.test(model, Food101)

[INFO] data loaded successfully


/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:488: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy          6.934653282165527
        test_loss            4.414797306060791
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_accuracy': 6.934653282165527, 'test_loss': 4.414797306060791}]

In [31]:
trainer.validate(model, Food101)

[INFO] data loaded successfully


Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      val_accuracy          7.0574259757995605
        val_loss             4.404572486877441
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_accuracy': 7.0574259757995605, 'val_loss': 4.404572486877441}]